In [1]:
from mayavi import mlab
mlab.init_notebook()

Notebook initialized with ipy backend.


In [224]:
from pyscf import gto, scf, lo
import numpy as np
from functools import reduce
from pyscf.lo.orth import pre_orth_ao_atm_scf
import ase, scipy
from pyscf import lo
import itertools as itl
import ase.visualize as av

T,F=True,False
np.set_printoptions(precision=2,suppress=True)

def calc(aseobj, bst='cc-pvdz', icab=F):
    zs = aseobj.numbers
    nh = (zs==1).sum()
    coords = aseobj.positions
    assert zs[0]>1 and np.all(zs[1:]==1) #zs = []; coords = []
    spin = sum(zs)%2
    atom = ''
    na = len(aseobj)
    for i in range(na):
        x,y,z = coords[i]
        ai = aseobj[i]
        atom += '%s %.8f %.8f %.8f;'%(ai.symbol, x,y,z)
    if icab:
        basis = {'H':'sto-3g'}
    else:
        basis = {'H':bst}
    for si in ['C','N','O','F']: basis[si] = bst
    mol = gto.M(atom=atom, basis=basis, verbose=0, spin=spin)
    mf = None
    if not icab:
        mf = scf.RHF(mol)
        mf.kernel()
    return mol, mf

def get_hao(mol):
    zs = mol.atom_charges()
    nh = (zs==1).sum()
    s1 = mol.intor_symmetric('int1e_ovlp')
    b1 = pre_orth_ao_atm_scf(mol)
    sb = reduce( np.dot, (b1.conjugate().T, s1, b1) )
    aolbs = mol.ao_labels(); nao = len(aolbs)
    sb_hx = sb[-nh:,:-nh] # overlap matrix H-X
    u,d,vh = np.linalg.svd(sb_hx, full_matrices=False, compute_uv=True)
    a1 = np.dot(vh.T, u.T)
    # now Schmidt orthogonalization
    n1 = nh
    n2 = nao - nh
    t = np.eye(n2)
    t[:,:nh] = a1
    for i in range(nh,n2):
        for j in range(i):
            cj = t[i,j] 
            t[:,i] -= cj*t[:,j]
        t[:,i] /= np.linalg.norm(t[:,i])
    for i in range(n2):
        csi = t[i,:6]
        so = ' '.join(['%10.2f '%si for si in csi])
        print(aolbs[i], so)
    return t

def get_new_dm1(mol, mf, t):
    cs = mf.mo_coeff
    return cs

def get_nho(m,bst='sto-3g',new_idx=None,debug=F):
    mol, _ = calc(m, bst=bst, icab=T)
    mf = scf.RHF(mol)
    mf.kernel()
    s = mol.intor_symmetric('int1e_ovlp_sph')
    iano=T#F
    if iano:
        A1 = pre_orth_ao_atm_scf(mol)
        s1 = reduce(np.dot, (A1.T,s,A1)) # under ANO basis
    else:
        A1 = np.eye(mol.nao)
        s1 = s
    if debug: print('s1=',s1)
    B1 = np.linalg.solve(A1,mf.mo_coeff)
    dm1 = reduce( np.dot, (B1, np.diag(mf.mo_occ), B1.T) ) ##
    if debug: print('dm1=',dm1)
    p1 = dm1 # reduce(np.dot, (s1, dm1, s1))
    zs = mol.atom_charges()
    nh = (zs==1).sum()
    e1,v1 = scipy.linalg.eigh(p1[:-nh,:-nh], s1[:-nh,:-nh])
    eigs1 = e1[::-1]; vs1 = v1[:,::-1]
    if debug: print('eigs=',eigs1)
    
    # exchange ao idx
    if new_idx is None:
        new_idx = np.arange(vs1.shape[0])
    # = [0,3,2,1,4]
    vs1u = vs1[:,new_idx]
    
    c1 = np.eye(mol.nao)
    c1[:-nh,:-nh] = vs1u # ANO basis

    a = np.linalg.solve(c1,B1)
    #if debug: 
    print('a=',a)
    return eigs1,vs1,a,c1

def print_c(aols, c1):
    nao = len(aols) 
    for i in range(nao):
        si = aols[i]
        for j in range(nao):
            si += '%6.2f '%c1[i,j]
        print(si)

def get_nho1(m,bst='sto-3g',iprt=F):
    mol, _ = calc(m, bst=bst, icab=T)
    zs = mol.atom_charges()
    nh = (zs==1).sum()
    aols = mol.ao_labels()
    aoidxs = mol.offset_ao_by_atom()[:, 2:]
    mf = scf.RHF(mol)
    mf.kernel()
    s = mol.intor_symmetric('int1e_ovlp')
    p = mf.make_rdm1() # reduce(np.dot, (s, mf.make_rdm1(), s))
    #ne = np.trace( np.dot(p,s) )
    #print('ne=',ne)
    na = len(zs)
    es = np.zeros(mol.nao); c = np.eye(mol.nao)
    for i in range(na):
        ib, ie = aoidxs[i]
        #if ie-ib==1: continue
        e, v = scipy.linalg.eigh(p[ib:ie,ib:ie], s[ib:ie,ib:ie])
        es[ib:ie] = e[::-1]
        c[ib:ie,ib:ie] = v[:,::-1]
    if iprt: print_c(aols,c)
    b1 = mf.mo_coeff
    a = np.linalg.solve(c,b1)
    #s1 = reduce(np.dot, (c.T,s,c))
    #if iprt: print_c(aols,a)
    occ = mf.mo_occ
    #if iprt: print('occ=',occ)
    dm = reduce(np.dot, (a, np.diag(occ), a.T))
    #print('new ne=', np.trace(np.dot(dm,s1)))
    #dm = reduce(np.dot, (a.T, np.diag(occ), a))
    return dm, c, a, aols

In [196]:
np.set_printoptions(precision=4,suppress=True)

bst = 'sto-3g' # 'cc-pvdz'
zs = [9,1]; coords = [[0.,0.,0],[0.,0.,0.98]]
m = ase.Atoms(zs,coords)
#av.view(m)
#eigs, vs, a, c = get_nho(m,bst=bst,debug=F)


In [197]:
m2 = m.copy()
m2.rotate(60, [1,1,1])

In [ ]:
#av.view(m)
idxs = list(itl.permutations([1,2,3]))
for i,idx in enumerate(idxs[0:1]):
    idx2 = [0]+list(idx)+[4]
    print('i,idx=',i,idx2)
    eigs2, vs2, a2, c2 = get_nho(m2,bst=bst,new_idx=[0]+list(idx)+[4], debug=F)
    #print('ddm: ', np.max(a2-a), np.min(a2-a), np.abs(a2)-np.abs(a))

In [225]:
bsts = ['sto-3g', 'cc-pvdz']
i = 1
bst = bsts[i]
dm1, c1, a1, aols = get_nho1(m,bst=bst,iprt=T)

0 F 1s     -1.18  -0.00  -0.00   0.07   0.01   0.00   0.00  -0.00   0.00   0.00  -0.00   0.00  -0.00   0.00   0.00 
0 F 2s     -0.82   0.00   0.00   0.14  -0.44  -1.33   0.01  -0.00   0.00   0.00  -0.00   0.02  -0.09   0.14   0.00 
0 F 3s      0.86   0.00  -0.00   0.64  -0.30   1.18  -0.02   0.00  -0.00  -0.01   0.00  -0.01   0.07  -0.12   0.00 
0 F 2px    -0.00   0.80  -0.15  -0.00   0.00  -0.00  -0.57  -0.00   0.00   0.01  -0.00  -0.56   0.02   0.13   0.00 
0 F 2py    -0.00   0.15   0.80   0.00  -0.00   0.00  -0.05   0.00   0.00   0.01   0.01  -0.07  -0.69  -0.41   0.00 
0 F 2pz    -0.04  -0.00   0.00  -0.57  -0.65   0.08   0.01   0.00  -0.00  -0.01   0.00  -0.14   0.38  -0.63   0.00 
0 F 3px    -0.00   0.30  -0.06  -0.00   0.00   0.00   0.78   0.00  -0.00  -0.04   0.01   0.76  -0.03  -0.18   0.00 
0 F 3py    -0.00   0.06   0.30   0.00  -0.00  -0.00   0.07  -0.00  -0.00  -0.01   0.01   0.10   0.95   0.56   0.00 
0 F 3pz    -0.01   0.00  -0.00  -0.15  -0.14   0.18  -0.07  -0.00   0.00

In [226]:
dm2, c2, a2, aols = get_nho1(m2,bst=bst,iprt=T)

0 F 1s     -1.18  -0.00   0.00   0.07   0.01  -0.00   0.00   0.00   0.00  -0.00   0.00  -0.00   0.00  -0.00   0.00 
0 F 2s     -0.82   0.00  -0.00   0.14  -0.44   1.32  -0.03   0.02  -0.00  -0.00   0.00   0.01  -0.00  -0.22   0.00 
0 F 3s      0.86  -0.00   0.00   0.64  -0.30  -1.18   0.03  -0.00   0.00   0.01  -0.00   0.01  -0.01   0.19   0.00 
0 F 2px    -0.03  -0.49   0.36  -0.38  -0.43   0.01   0.10  -0.15  -0.01  -0.00  -0.01   0.01   0.02   0.76   0.00 
0 F 2py     0.01  -0.63  -0.44   0.19   0.22   0.06   0.79  -0.03   0.00  -0.00  -0.00   0.03  -0.00  -0.09   0.00 
0 F 2pz    -0.03   0.18  -0.58  -0.38  -0.43  -0.14  -0.02  -0.75  -0.01   0.02  -0.00   0.02  -0.01  -0.19   0.00 
0 F 3px    -0.01  -0.18   0.13  -0.10  -0.09  -0.20  -0.12   0.27   0.03   0.01   0.01   0.01  -0.07  -1.06   0.00 
0 F 3py     0.00  -0.23  -0.16   0.05   0.05   0.01  -1.10   0.01   0.01  -0.00   0.00  -0.07   0.02   0.13   0.00 
0 F 3pz    -0.01   0.07  -0.22  -0.10  -0.09   0.00   0.05   1.09   0.01

In [229]:
for c in [c1,c2]:
    print(np.linalg.norm(c[3:6],axis=0))

[0.04 0.82 0.82 0.57 0.65 0.08 0.57 0.   0.   0.02 0.01 0.58 0.79 0.76
 0.  ]
[0.04 0.82 0.82 0.57 0.65 0.15 0.8  0.76 0.01 0.02 0.01 0.04 0.02 0.79
 0.  ]


In [230]:
for c in [c1,c2]:
    print(np.linalg.norm(c[6:9],axis=0))

[0.01 0.3  0.3  0.15 0.14 0.18 0.79 0.   0.   0.04 0.02 0.81 1.1  1.09
 0.  ]
[0.01 0.3  0.3  0.15 0.14 0.2  1.11 1.12 0.04 0.02 0.01 0.07 0.07 1.09
 0.  ]


In [212]:
print_c(aols, a2-a1)

0 F 1s      0.00  -0.00  -0.00   0.00   0.00   0.00  -0.00   0.00   0.00   0.00  -0.00   0.00   0.00   0.00   0.00 
0 F 2s     -0.00  -0.00  -0.00  -0.55  -1.72   0.00   0.25   0.15  -0.00  -0.00  -0.00  -0.00   0.00   0.00  -0.00 
0 F 3s      0.00   0.00  -0.00  -0.24  -0.75  -0.00   0.15   0.09   0.00   0.00  -0.01  -0.00  -0.00  -0.00   0.00 
0 F 2px     0.00  -0.00  -0.00  -0.00   0.00  -0.00   0.00  -0.00   0.00   0.00  -0.00   0.00   0.00   0.00   0.00 
0 F 2py     0.00  -0.00   0.00   0.00   0.00   0.00  -0.00   0.00  -0.00  -0.00   0.00  -0.00  -0.00  -0.00  -0.00 
0 F 2pz    -1.03   0.18  -0.34  -0.02  -0.00   0.43  -0.10   0.09   0.19  -2.01  -0.01   0.00  -0.00   0.00  -0.53 
0 F 3px     0.02   0.01   0.07  -0.01  -0.28  -0.16  -1.10  -1.10   0.38   0.32  -0.07   0.06  -0.01   0.02  -0.56 
0 F 3py    -0.00   0.02   0.14  -0.08   0.06  -0.26  -0.06   0.59   0.86   0.48   0.02  -0.01   0.17   0.98   0.25 
0 F 3pz     0.00   0.00   0.01   0.00   0.01  -0.02   0.02   0.00   0.03

In [213]:
ddm = dm2-dm1
print_c(aols, ddm)#[:5,:5])

0 F 1s     -0.00   0.00  -0.00  -0.00  -0.00   1.77  -0.04  -0.00  -0.00  -0.01   0.00  -0.01   0.05  -0.21  -0.00 
0 F 2s      0.00   0.00   0.00  -0.00  -0.00   0.03   0.04   0.02  -0.02   0.02  -0.01  -0.22  -0.03   0.20   0.00 
0 F 3s     -0.00   0.00   0.00  -0.00  -0.00  -0.03   0.18  -0.20  -0.00  -0.01  -0.01   0.03  -0.29  -0.39  -0.00 
0 F 2px    -0.00  -0.00  -0.00  -0.00  -0.00   0.38  -0.08  -0.17  -0.01  -0.00  -0.00   0.04  -0.09   0.25  -0.00 
0 F 2py    -0.00  -0.00  -0.00  -0.00  -0.00  -0.10  -0.06  -0.13  -0.01  -0.01  -0.00   0.03  -0.06   0.23   0.00 
0 F 2pz     1.77   0.03  -0.03   0.38  -0.10  -0.01  -0.02  -0.03  -0.00  -0.00  -0.00  -0.01  -0.00  -0.04  -0.22 
0 F 3px    -0.04   0.04   0.18  -0.08  -0.06  -0.02   0.03   0.00  -0.00   0.00  -0.00  -0.02  -0.01   0.01   0.06 
0 F 3py    -0.00   0.02  -0.20  -0.17  -0.13  -0.03   0.00   0.06   0.00   0.00  -0.00   0.00  -0.00   0.00   0.11 
0 F 3pz    -0.00  -0.02  -0.00  -0.01  -0.01  -0.00  -0.00   0.00   0.00

In [ ]:
# write HAO
import interfaces._pyscf as pscf
#reload(pscf)
oo = pscf.io(mol)
c1 = np.eye(mol.nao)
c1[:-1,:-1] = vs1
orig, cell, dt = oo.orbital(c1, grids=[100,100,100], label=None)#'ch4-carbon')

In [23]:
a

array([[ 0.0268, -0.0889,  0.0441,  0.7229,  0.6835,  0.0356],
       [ 0.9204, -0.3287,  0.1443, -0.1466,  0.0682,  0.0844],
       [ 0.3847,  0.6978, -0.3602,  0.3973, -0.3262, -0.3143],
       [ 0.0683,  0.4785, -0.2425, -0.5459,  0.6494, -0.2042],
       [ 0.0061, -0.1743, -0.6489,  0.    , -0.    ,  0.8589],
       [-0.0051,  0.1453,  0.541 , -0.    ,  0.    ,  1.0334]])

In [32]:
#from ase.io.cube import read_cube_data
#data, atoms = read_cube_data('ch4-carbon_01.cube')
import visualization.mayavi as mv
_atoms = mv.draw_molecule(m, dt[0], cell, orig)
#_atoms

 ## found 0 non-covalent bonds
